## Data Loading

In [ ]:
# import libraries

import pandas as pd
import numpy as np
# ==== Imports ====
import re, os, plotly.express as px
import statsmodels.formula.api as smf
import statsmodels.api as sm
from math import sqrt

# Hypothesis 1: Lifestyle Habits and Diabetes

In [ ]:
# ==== Load a single base_df ====
if 'df' in globals() and isinstance(df, pd.DataFrame):
    base_df = df.copy()
elif 'df_brfss' in globals() and isinstance(df_brfss, pd.DataFrame):
    base_df = df_brfss.copy()
elif os.path.exists('diabetes_binary_5050split_health_indicators_BRFSS2015.csv'):
    base_df = pd.read_csv('diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
else:
    raise RuntimeError("No df/df_brfss found and CSV not present. Load your data first.")

# standardize column names
base_df.columns = (base_df.columns
                   .str.strip().str.lower()
                   .str.replace(' ', '_').str.replace('-', '_'))

# ==== Diabetes: create diabetes_binary (0/1) robustly ====
def to_diabetes_binary(s):
    num = pd.to_numeric(s, errors='coerce')
    if num.notna().any():
        return (num >= 1).astype('Int64')
    m = {'1':1,'0':0,'yes':1,'no':0,'true':1,'false':0,'y':1,'n':0,
         'positive':1,'negative':0,'diabetic':1,'non-diabetic':0}
    s2 = s.astype(str).str.strip().str.lower().replace({'1.0':'1','0.0':'0'})
    return s2.map(m).astype('Int64')

if 'diabetes_binary' in base_df.columns:
    base_df['diabetes_binary'] = to_diabetes_binary(base_df['diabetes_binary'])
elif 'diabetes' in base_df.columns:
    base_df['diabetes_binary'] = to_diabetes_binary(base_df['diabetes'])
else:
    raise KeyError("Expected a 'diabetes' or 'diabetes_binary' column.")

# ==== Education: rebuild clean ordered categories ====
edu_order = [
    "Never attended/Kindergarten",
    "Elementary",
    "Some high school",
    "High school grad",
    "Some college/Tech",
    "College grad",
]

def map_education(series):
    # Try numeric codes first
    num = pd.to_numeric(series, errors='coerce')
    if num.notna().any():
        map_1_6 = {1:edu_order[0],2:edu_order[1],3:edu_order[2],4:edu_order[3],5:edu_order[4],6:edu_order[5]}
        map_0_5 = {0:edu_order[0],1:edu_order[1],2:edu_order[2],3:edu_order[3],4:edu_order[4],5:edu_order[5]}
        uniq = set(num.dropna().unique())
        if uniq.issubset(set(map_1_6)):
            lab = num.map(map_1_6)
        elif uniq.issubset(set(map_0_5)):
            lab = num.map(map_0_5)
        else:
            # Try to coerce to 1..6 if it's near that
            if min(uniq) >= 1 and max(uniq) <= 6: lab = num.map(map_1_6)
            elif min(uniq) >= 0 and max(uniq) <= 5: lab = num.map(map_0_5)
            else: lab = pd.Series([np.nan]*len(series))
    else:
        # Text labels → regex map
        txt = series.astype(str).str.strip().str.lower()
        alias = {
            r'never|kindergarten|no school': edu_order[0],
            r'elementary|grade school|primary': edu_order[1],
            r'some high\s*school|partial high|years? of high': edu_order[2],
            r'high school.*grad|hs grad|secondary': edu_order[3],
            r'some college|tech|technical|vocational|community college': edu_order[4],
            r'college grad|bachelor|undergrad|university degree|degree holder': edu_order[5],
        }
        def map_by_regex(v):
            for pat, lab in alias.items():
                if re.search(pat, v): return lab
            return np.nan
        lab = txt.apply(map_by_regex)

    return pd.Categorical(lab, categories=edu_order, ordered=True)

# If 'education' exists but is broken/all-NA, or doesn't exist, try to rebuild
if 'education' in base_df.columns and not base_df['education'].isna().all():
    # respect existing if valid; otherwise rebuild from it
    base_df['education'] = map_education(base_df['education'])
else:
    # try a common raw field name from CSV
    raw_candidates = [c for c in base_df.columns if c in ['education','educ','educa','education_level','edu_level']]
    if raw_candidates:
        base_df['education'] = map_education(base_df[raw_candidates[0]])
    else:
        # fallback: create NA categorical so plots warn helpfully
        base_df['education'] = pd.Categorical([np.nan]*len(base_df), categories=edu_order, ordered=True)

# ==== Utility helpers ====
def prop_ci_wilson(success, n, z=1.96):
    if n == 0: return (np.nan, np.nan)
    phat = success / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n))/denom
    half = (z * np.sqrt((phat*(1-phat)+z**2/(4*n))/n)) / denom
    return center - half, center + half

def prevalence(df, by):
    g = df.groupby(by)['diabetes_binary']
    out = g.mean().rename('prevalence').to_frame()
    out['n'] = g.count()
    out['cases'] = g.sum()
    cis = out.apply(lambda r: pd.Series(prop_ci_wilson(r['cases'], r['n']), index=['ci_lo','ci_hi']), axis=1)
    return pd.concat([out, cis], axis=1).reset_index()

def odds_ratio_table(model):
    params = model.params
    conf = model.conf_int()
    return pd.DataFrame({
        'term': params.index,
        'OR': np.exp(params.values),
        'CI_lo': np.exp(conf[0].values),
        'CI_hi': np.exp(conf[1].values),
        'p': model.pvalues.values
    })

print("Ready. diabetes_binary value counts:\n", base_df['diabetes_binary'].value_counts(dropna=False))
print("Education NA%:", float(base_df['education'].isna().mean()))

In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# --- assumes you already have base_df and prevalence(df, by) defined ---

# 1) Causes that exist
causes_all = ['smoker', 'physactivity', 'hvyalcoholconsump', 'fruits', 'veggies']
causes = [c for c in causes_all if c in base_df.columns]
if not causes:
    raise ValueError("None of the requested cause columns are in base_df.")

# 2) Pre-compute prevalence tables
data_map = {}
for c in causes:
    prev = prevalence(base_df, c)  # columns: [c, prevalence, n, cases, ci_lo, ci_hi]
    # Nicer labels if binary
    if prev[c].dropna().isin([0, 1]).all():
        prev[c] = prev[c].map({0: "No", 1: "Yes"})
    data_map[c] = prev

# 3) Initial figure
first = causes[0]
prev0 = data_map[first]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=prev0[first],
    y=prev0['prevalence'],
    error_y=dict(
        type='data',
        array=(prev0['ci_hi'] - prev0['prevalence']),
        arrayminus=(prev0['prevalence'] - prev0['ci_lo'])
    ),
    text=prev0['prevalence'],
    texttemplate='%{text:.1%}',
    textposition='outside',
    hovertemplate=(
        # NOTE: This is a plain string (NOT an f-string), so %{x} is preserved for Plotly.
        "Category = %{x}<br>"
        "Prevalence = %{y:.1%}<br>"
        "n = %{customdata[0]}<br>"
        "cases = %{customdata[1]}<extra></extra>"
    ),
    customdata=np.stack([prev0['n'], prev0['cases']], axis=-1),
))

fig.update_layout(
    title=f"Diabetes Prevalence by {first.title()}",
    yaxis=dict(range=[0, 1], tickformat=".0%", title="Prevalence"),
    xaxis_title=first.title(),
    margin=dict(l=60, r=20, t=70, b=60),
)

# 4) Dropdown buttons
buttons = []
for c in causes:
    prev = data_map[c]
    x_vals = prev[c]  # already converted to Yes/No if binary
    buttons.append(dict(
        method="update",
        label=c,
        args=[
            # Trace updates
            {
                "x": [x_vals],
                "y": [prev['prevalence']],
                "text": [prev['prevalence']],
                "error_y": [dict(
                    type='data',
                    array=(prev['ci_hi'] - prev['prevalence']),
                    arrayminus=(prev['prevalence'] - prev['ci_lo'])
                )],
                "customdata": [np.stack([prev['n'], prev['cases']], axis=-1)],
            },
            # Layout updates
            {
                "title": f"Diabetes Prevalence by {c.title()}",
                "xaxis": {"title": c.title(), "categoryorder": "array",
                          "categoryarray": list(x_vals.astype(str))},
                "yaxis": {"range": [0, 1], "tickformat": ".0%"},
            }
        ]
    ))

fig.update_layout(
    updatemenus=[dict(
        buttons=buttons,
        direction="down",
        x=1.0, xanchor="right",
        y=1.15, yanchor="top",
        showactive=True
    )]
)

fig.show()

In [ ]:
for v in ['fruits','veggies']:
    if v not in base_df.columns: 
        continue
    s = base_df[v]
    num = pd.to_numeric(s, errors='coerce')
    if num.notna().sum() > 0 and num.nunique() > 6:
        base_df[f'{v}_bin'] = pd.qcut(num, 5, duplicates='drop')
        by = f'{v}_bin'
    else:
        # assume binary; treat 0 as "Low", 1 as "High"
        base_df[f'{v}_bin'] = s.replace({0:'Low', 1:'High'})
        by = f'{v}_bin'
    prev = prevalence(base_df, by)
    fig = px.line(prev, x=by, y='prevalence', markers=True,
                  title=f"Diabetes Prevalence by {v.title()} (dose/level)")
    fig.update_yaxes(range=[0,1], tickformat=".0%")
    fig.show()

In [ ]:
need = ['smoker','physactivity','hvyalcoholconsump','fruits','veggies']
avail = [c for c in need if c in base_df.columns]
if set(['smoker','physactivity']).issubset(avail):
    # Define risk: smoker==1, physactivity==0, hvyalcoholconsump==1, fruits==0, veggies==0
    rb = pd.DataFrame(index=base_df.index)
    if 'smoker' in avail: rb['smoker_risk'] = (base_df['smoker'] == 1).astype(int)
    if 'physactivity' in avail: rb['inactivity_risk'] = (base_df['physactivity'] == 0).astype(int)
    if 'hvyalcoholconsump' in avail: rb['alcohol_risk'] = (base_df['hvyalcoholconsump'] == 1).astype(int)
    if 'fruits' in avail: rb['fruit_risk'] = (base_df['fruits'] == 0).astype(int)
    if 'veggies' in avail: rb['veg_risk'] = (base_df['veggies'] == 0).astype(int)
    base_df['risk_behaviors'] = rb.sum(axis=1)
    prev = prevalence(base_df, 'risk_behaviors')
    fig = px.line(prev, x='risk_behaviors', y='prevalence', markers=True,
                  title="Diabetes Prevalence vs Number of Risky Behaviors (0–5)")
    fig.update_yaxes(range=[0,1], tickformat=".0%")
    fig.show()

In [ ]:
predictors = [c for c in ['smoker','physactivity','hvyalcoholconsump','fruits','veggies'] if c in base_df.columns]
controls = [c for c in ['age','sex','bmi','income'] if c in base_df.columns]
formula = 'diabetes_binary ~ ' + ' + '.join(predictors + controls)

m = smf.logit(formula, data=base_df.dropna(subset=['diabetes_binary'] + predictors + controls)).fit(disp=False)
or_tab = odds_ratio_table(m)

plot_df = or_tab[or_tab['term']!='Intercept'].copy()
fig = px.scatter(plot_df, x='OR', y='term', error_x='CI_hi', title='Adjusted ORs (Lifestyle + Controls)')
fig.update_traces(error_x=dict(type='data',
                               array=plot_df['CI_hi']-plot_df['OR'],
                               arrayminus=plot_df['OR']-plot_df['CI_lo']))
fig.add_vline(x=1, line_dash='dot'); fig.update_layout(xaxis_type='log'); fig.show()